In [2]:
import numpy as np
import os
dtype = np.float32

In [3]:
import torch
import torchvision
# import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

import glob
import random
import json
import os
def contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

/home/jgryu/miniconda3/envs/RD/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### train val 나눠서 path 저장

In [5]:
# dataset_path = '/home/jgryu/Weight_compression/Wparam_dataset/model_param_tensor/meta-llama/Meta-Llama-3-8B'
dataset_path = '/home/jgryu/Weight_compression/Wparam_dataset/model_param_tensor/meta-llama/Llama-2-7b-hf'
save_path = '/home/jgryu/Weight_compression/Wparam_dataset/path_json/'

tenosr_path_list = glob.glob(f'{dataset_path}/**/*.npy', recursive=True)

print(f'all : {len(tenosr_path_list)}')
# filters = ['meta-llama-3-8b', 'attn']
filters = ['llama-2-7b', 'attn']

random.seed(100)

data = {'filters': filters, 'path_list': []}

for path in tenosr_path_list:
    if contains_all_substrings(path.lower(), filters):
        data['path_list'].append(path)
        
print(len(data['path_list']))


random.shuffle(data['path_list'])

split_index = int(0.8 * len(data['path_list']))
train_data = data['path_list'][:split_index]
validation_data = data['path_list'][split_index:]

file_path_train = os.path.join(save_path, '_'.join(filters) + '_train.json')
file_path_validation = file_path_train.replace('train', 'val')

with open(file_path_train, 'w') as json_file:
    json.dump(train_data, json_file, indent=4)
with open(file_path_validation, 'w') as json_file:
    json.dump(validation_data, json_file, indent=4)

all : 290
128


In [ ]:
import glob
import random
import json
def contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

save_path = '/home/jgryu/Weight_compression/model_param_dataset/'
dataset_path = '/home/jgryu/Weight_compression/model_param_tensor/'

tenosr_path_list = glob.glob(f'{dataset_path}/**/*.npy', recursive=True)

print(f'all : {len(tenosr_path_list)}')

data = {'attn':[], 'mlp':[], 'embed':[], 'layernorm':[]}   

random.seed(100)

for weight_type in ['attn', 'mlp', 'embed', 'layernorm']: 
    for path in tenosr_path_list:
        if contains_all_substrings(path.lower(), [weight_type]):
            data[weight_type].append(path)
    print(f'{weight_type} : {len(data[weight_type])}')


    random.shuffle(data[weight_type])

    split_index = int(0.8 * len(data[weight_type]))
    train_data = data[weight_type][:split_index]
    validation_data = data[weight_type][split_index:]

    file_path_train = os.path.join(save_path, f'{weight_type}_tensor_path_train.json')
    file_path_validation = file_path_train.replace('train', 'val')

    with open(file_path_train, 'w') as json_file:
        json.dump(train_data, json_file, indent=4)
    with open(file_path_validation, 'w') as json_file:
        json.dump(validation_data, json_file, indent=4)

In [ ]:
# ## 
# import numpy as np
# json_file = '/home/jgryu/Weight_compression/Wparam_dataset/path_json/meta-llama-3-8b_mlp_val.json'
# save_path = '/home/jgryu/Weight_compression/Wparam_dataset/transformer_dataset'
# dim = 1024
# length = 128

# with open(json_file, 'r') as f:
#     tensor_path_list = json.load(f)

# data = []
# for tensor_path in tensor_path_list:
#     t = np.load(tensor_path)
#     data.append(t.reshape(-1, length, dim))
    
# data = np.concatenate(data, axis = 0)
# print(data.shape)
# path = os.path.join(save_path, json_file.split('/')[-1].replace('.json', ''))
# np.save(path, data)



(8960, 128, 1024)


### 1. Model tensor를 따로 npy로 저장

In [ ]:
import numpy as np
import os
dtype = np.float32

import torch
import torchvision
# import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

def get_ckpt_path(path, branch = 'main'):
    if not os.path.isdir(os.path.join(path, 'snapshots')):
        return None
    branch_file =  os.path.join(path, 'refs', branch)
    with open(branch_file, 'r', encoding='utf-8') as file:
        revision = file.read()
    return os.path.join(path, 'snapshots', revision)

def check_contains_any_substrings(string, substrings):
    return any(substring in string for substring in substrings)

model_zoo_path = '/home/jgryu/Weight_compression/Wparam_dataset/model_zoo/huggingface'
# model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b', '13b', 'mini', 'small']
# model_filter = ['0.5b','1.5b','2b','3b', '7b', '8b', '9b','mini']
# model_filter = ['-13b', '-small']
model_filter = ['meta-llama-3-8b', 'meta-llama-3.1-8b']

model_list = os.listdir(model_zoo_path)
print()
ckpt_path_list = []
for ck in model_list:
    if check_contains_any_substrings(ck.lower(), model_filter):
        ckpt_path = get_ckpt_path(os.path.join(model_zoo_path, ck))
        if ckpt_path is not None:
            ckpt_path_list.append(ckpt_path)

print(len(ckpt_path_list))

save_dir = '/home/jgryu/Weight_compression/Wparam_dataset/model_param_tensor'

for ckpt_path in ckpt_path_list:
    model_name = ckpt_path.split('/')[-3]
    print(model_name)
    model_name = model_name.split('--')
    
    # save_path = os.path.join(save_dir, model_name[1], model_name[2])

    # if os.path.isdir(save_path) and bool(os.listdir(save_path)):
    #     print('### Skip ###')
    #     continue

    try:
        model = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True, trust_remote_code=True)
        
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        state_dict = model.state_dict()
        for k, v in state_dict.items():
            np.save(os.path.join(save_path, k.replace(".", "-")), v)
            print(k, v.shape)
            
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Fail load model from {ckpt_path}")

